In [ ]:
# 📄 CSV yükleyici
from google.colab import files
uploaded = files.upload()

Chatbot Using Mistral-7B-Instruct-v0.1

In [ ]:
!pip install -q transformers sentence-transformers gradio accelerate
!pip install -q transformers datasets peft accelerate bitsandbytes trl

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import torch
import gradio as gr
import pandas as pd
import textwrap

print("Is CUDA available? ", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))

# 1. Klinik notlarını yükle
df = pd.read_csv("clinical_notes.csv")
df['not'] = df.apply(lambda row: f"{row['hasta_adi']}, Etiket: {row['etiket']}. {row['not']}", axis=1)


# 2. Embedding modeli
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
note_texts = df['not'].tolist()
note_embeddings = embedder.encode(note_texts, convert_to_tensor=True)

# 3. HF API uzerinden calisan LLM Modeli
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,

)

# 4. Cevap fonksiyonu
def answer_question(question):
    # Step 1: Embed the user question
    q_embed = embedder.encode(question, convert_to_tensor=True)

    # Step 2: Semantic search over notes
    hits = util.cos_sim(q_embed, note_embeddings)[0].topk(3)
    relevant_notes = [note_texts[idx] for idx in hits[1]]

    # Step 3: Check similarity score — are they actually related?
    top_score = hits[0][0].item()
    similarity_threshold = 0.4  # Tune if needed

    if top_score < similarity_threshold:
        # No relevant notes — use LLM as a general chatbot
        print("⚠️ No relevant medical notes found — switching to open chat mode.")

        prompt = textwrap.dedent(f"""\
        Sen arkadaş canlısı bir yapay zeka asistanısın. Kullanıcının sorusuna veya mesajına doğal ve samimi bir şekilde cevap ver.

        Kullanıcı: {question}
        Asistan:""")

    else:
        # RAG: medical mode
        context = "\n".join([f"- {note}" for note in relevant_notes])

        prompt = textwrap.dedent(f"""\
        Sen bir klinik yapay zeka asistanısın. Aşağıda bazı hasta notları var. Bu notlara dayanarak soruya açıklayıcı bir yanıt ver.

        Soru: {question}
        Klinik Notlar:
        {context}

        Yanıt:""")

    # Step 4: Generate model response
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Step 5: Clean output
    if "Yanıt:" in decoded:
        response = decoded.split("Yanıt:")[-1].strip()
    elif "Cevap:" in decoded:
        response = decoded.split("Cevap:")[-1].strip()
    else:
        response = decoded.strip()

    return response


# 5. Gradio chat arayüzü
gr.ChatInterface(
    fn=lambda message, history: answer_question(message),
    title="Klinik Notlara Dayalı AI Yanıtlayıcı",
    description="Yüklenen klinik notlara göre LLM destekli cevap üretir."
).launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.1 MB/s eta 0:00:00
